# Part 1 - That's a Bingo!

I'm going to start this one by writing a class and some functions that contains some useful stuff, and then start actually importing the data afterwards. The first decision that needs to be made is how to store the bingo boards, and the board state (i.e. what tiles have been called). The board state doesn't need to be stored for each board between rounds, it's probably easier to recreate it from scratch each time to see if a board is winning.

The most obvious way to store the boards is just as 5 lists of 5 elements each. Each element representing one number across the board, and each list representing one line of the board, top to bottom. The board state can be stored as an equivalent array of booleans. The overall set of boards can just be a list of these 

In [10]:
import math # needed for square root function to find size of bingo board

class Board:
    def __init__(self, values):
        value_index = 0
        self.values = []
        size = int(math.sqrt(len(values)))
        for y in range(size):
            line = []
            for x in range(size):
                line.append(values[value_index])
                value_index += 1
            self.values.append(line)
        self.selected = [[0 for col in range(size)] for row in range(size)]

    
    def __str__(self):
        return_string = []
        for i in range(len(self.values)):
            return_string.append(str(self.values[i]))
        
        return str("\n".join(return_string))
        
    
    def select_squares(self, selected_numbers):
        for y, line in enumerate(self.values):
            for x, element in enumerate(line):
                if element in selected_numbers:
                    self.selected[y][x] = 1
                else:
                    self.selected[y][x] = 0
        
    
    def check_win(self):
        # check for across win and return true if present
        for line in self.selected:
            if sum(line) == len(line):
                return True
        
        # check for vertical win and return true if present
        for i in range(len(self.selected[0])):
            selected_squares = 0
            for line in self.selected:
                selected_squares += line[i]
            if selected_squares == len(self.selected):
                return True
            
        # if neither across nor vertical win, return false    
        return False
    
def return_winner(selected_numbers, boards):
    
    called_numbers = []

    for i in range(len(selected_numbers)):
        called_numbers.append(selected_numbers[i])
        for board in boards:
            board.select_squares(called_numbers)
            if board.check_win():
                return board, int(called_numbers[-1])

Then, I just need to parse the input file to load the boards, and use the new functions to find the winning board. A little bit of processing to find the unselected numbers, and I have the solve

In [12]:
with open('day4_input.txt') as file:
    lines = file.readlines()

selected_numbers = [int(i) for i in lines[0].strip().split(",")]

del lines[0:2]

boards = []

while lines:
    board_values = []
    for i in range(5):
        board_values += [int(i) for i in lines[i].strip().split()]
    boards.append(Board(board_values))
    del lines[0:6]
    
winner, winning_number = return_winner(selected_numbers, boards)

unselected_values_sum = 0

for y, line in enumerate(winner.values):
    for x, element in enumerate(line):
        if winner.selected[y][x] == 0:
            unselected_values_sum += winner.values[y][x]
            
print(f"The winning board is: \n{winner}\n")
print(f"The winning number is {winning_number}\n")
print(f"The sum of the numbers not selected from the winning board is {unselected_values_sum}\n")
print(f"The answer is {winning_number * unselected_values_sum}")

The winning board is: 
[86, 80, 77, 18, 87]
[79, 93, 52, 17, 20]
[30, 68, 48, 12, 91]
[25, 98, 13, 9, 47]
[45, 73, 97, 15, 59]

The winning number is 73

The sum of the numbers not selected from the winning board is 806

The answer is 58838


# Part 2 - Squid's game

We should be able to do almost the same thing, but we need a new function `return_loser` which will return the last winning board, as well as the winning number. This function goes through the input numbers until all the boards have won, and then steps one input back to find a board that has won on the current round, but not the previous round

In [23]:
def return_loser(selected_numbers, boards):
    
    called_numbers = []
    
    for i in range(len(selected_numbers)):
        winners = 0
        called_numbers.append(selected_numbers[i])
        for board in boards:
            board.select_squares(called_numbers)
            if board.check_win():
                winners += 1
        if winners == len(boards):
            previous_numbers = called_numbers[0:-1]
            winning_number = called_numbers[-1]
            break
            
    for board in boards:
        board.select_squares(previous_numbers)
        prev_win = board.check_win()
        
        board.select_squares(called_numbers)
        current_win = board.check_win()
        
        
        if current_win and not prev_win:
            return board, int(winning_number)

Almost all the code from before can be resued here to call the functions and determine the losing board

In [24]:
with open('day4_input.txt') as file:
    lines = file.readlines()

selected_numbers = [int(i) for i in lines[0].strip().split(",")]

del lines[0:2]

boards = []

while lines:
    board_values = []
    for i in range(5):
        board_values += [int(i) for i in lines[i].strip().split()]
    boards.append(Board(board_values))
    del lines[0:6]
    
loser, winning_number = return_loser(selected_numbers, boards)

unselected_values_sum = 0

for y, line in enumerate(loser.values):
    for x, element in enumerate(line):
        if loser.selected[y][x] == 0:
            unselected_values_sum += loser.values[y][x]
            
print(f"The last winning board is: \n{loser}\n")
print(f"The winning number is {winning_number}\n")
print(f"The sum of the numbers not selected from the winning board is {unselected_values_sum}\n")
print(f"The answer is {winning_number * unselected_values_sum}")

The last winning board is: 
[17, 16, 10, 68, 76]
[2, 45, 94, 29, 40]
[1, 54, 60, 66, 93]
[0, 13, 42, 39, 70]
[6, 82, 46, 74, 43]

The winning number is 46

The sum of the numbers not selected from the winning board is 136

The answer is 6256
